```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016, 2017, 2018  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Simple SWE Animations 

Particular focus on the influence of different boundary conditions

## Technical Initializations

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
#%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec

import os
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import datetime
import sys

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../')))

#Set large figure sizes
#rc('figure', figsize=(16.0, 12.0))
#rc('animation', html='html5')
plt.rcParams["animation.html"] = "jshtml"

#Import our simulator
from SWESimulators import FBL, CTCS, KP07, CDKLM16, PlotHelper, Common, WindStress, IPythonMagic
#Import initial condition and bathymetry generating functions:
from SWESimulators.BathymetryAndICs import *

In [ ]:
%setup_logging --out compareschemes2d.log
%cuda_context_handler gpu_ctx

Console logger using level INFO
File logger disabled
Python version 3.7.8 | packaged by conda-forge | (default, Jul 31 2020, 01:53:57) [MSC v.1916 64 bit (AMD64)]
Registering gpu_ctx in user workspace
PyCUDA version 2019.1.2
CUDA version (11, 1, 0)
Driver version 11010
Using 'Quadro T2000' GPU
Created context handle <2115581098960>


In [ ]:
#Create output directory for images
imgdir='images_' + datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
os.makedirs(imgdir)
print("Saving images to " + imgdir)

Saving images to images_2020_12_01-16_20_20


## Animation Utilities 

`sim_animation` generates an animation of the SWE Simulation

In [ ]:

def sim_animation(simulator, T):
    eta1, u1, v1 = sim.download(interior_domain_only=True)
    
    #Create figure and plot initial conditions
    fig = plt.figure(figsize=(12, 8))
    domain_extent = [0, sim.nx*sim.dx, 0, sim.ny*sim.dy]
    
    ax_eta = plt.subplot(1,3,1)
    sp_eta = ax_eta.imshow(eta1, interpolation="none", origin='lower', vmin=-0.5, vmax=0.5, extent=domain_extent)
    
    ax_u = plt.subplot(1,3,2)
    sp_u = ax_u.imshow(u1, interpolation="none", origin='lower', vmin=-1.5, vmax=1.5, extent=domain_extent)
    
    ax_v = plt.subplot(1,3,3)
    sp_v = ax_v.imshow(v1, interpolation="none", origin='lower', vmin=-1.5, vmax=1.5, extent=domain_extent)
    
    #Helper function which simulates and plots the solution
    def animate(i):
        if (i>0):
            t = sim.step(10.0)
        else:
            t = 0.0
        eta1, u1, v1 = sim.download(interior_domain_only=True)
        
        #Update plots
        fig.sca(ax_eta)
        sp_eta.set_data(eta1)
        
        fig.sca(ax_u)
        sp_u.set_data(u1)
        
        fig.sca(ax_v)
        sp_v.set_data(v1)
        
        fig.suptitle("Time = {:04.0f} s ({:s})".format(t, sim.__class__.__name__), fontsize=18)
        print(".", end='')

    #Matplotlib for creating an animation
    anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
    plt.close(fig)
    return anim

## Set-up of Simulation

-   Domain and Parameters

In [ ]:
# Set initial conditions common to all simulators
sim_args = {
"gpu_ctx": gpu_ctx,
"nx": 100, "ny": 200,
"dx": 200.0, "dy": 200.0,
"dt": 0.0,
"g": 9.81,
"f": 0.012,
#"coriolis_beta": 1.0e-6,
"coriolis_beta": 0.0,
"r": 0.0
}

ghost_cells = np.array([2,2,2,2]) # north, east, south, west
dataShape = (sim_args["ny"] + ghost_cells[0]+ghost_cells[2], 
             sim_args["nx"] + ghost_cells[1]+ghost_cells[3])

-   Boundary Conditions

    - Boundary Conditions Type 
    
    There are 4 different types of boundary conditions available: 1 = Wall, 2 = Periodic (requires same for opposite boundary as well), 3 = Open Boundary with Flow Relaxation Scheme, 4 = Open linear interpolation (Options 3 and 4 are of sponge type), see `Common.BoundaryConditions`.

    The sponge area is outer ring of the interior domain, where the relaxation/interpolation is applied after each step of solving the SWE. 

    <img src="pics/SpongeBoundarySteps.png" width = "350">

    -   Boundary Conditions Data

By default, the boundary condition data in the ghost cells is set as 0, see `Common.SingleBoundaryConditionData`. 


In [ ]:
boundary_conditions_type = {"north":3, "east":3, "south":3, "west":3}
#sponge_cells = {"north":20, "east":20, "south":20, "west":20}
sponge_cells = {"north":1, "east":1, "south":1, "west":1}
boundary_conditions = Common.BoundaryConditions(**boundary_conditions_type, spongeCells=sponge_cells)


-   Initial Conditions

For different "bumps", see `BathymetryAndICs`

In [ ]:
H = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32) * 60.0
eta0 = np.zeros(dataShape, dtype=np.float32)
u0 = np.zeros(dataShape, dtype=np.float32)
v0 = np.zeros(dataShape, dtype=np.float32)

#Create bump in to lower left of domain for testing
addBump(eta0, sim_args["nx"], sim_args["ny"], sim_args["dx"], sim_args["dy"], 0.5, 0.5, 1.0, ghost_cells, 1.0)



-   Simulator

Our standard CDKLM16 method is chosen. 

By the choise of `dt=0.0` the time step size is automatically set to 0.8 of the CFL condition before the simulation starts. To further time step size adjustments the switch `update_dt = True` can be enabled in the `sim.step(T)` call which is hidden in the `sim_animation` function. (For later use in data assimilation the time step size is adjusted every now and then by default.)

In [ ]:
#Initialize simulator
cdklm_args = {"H": H, "eta0": eta0, "hu0": u0, "hv0": v0, "rk_order": 2}
sim = CDKLM16.CDKLM16(**cdklm_args, **sim_args, boundary_conditions=boundary_conditions)

#Run a simulation and plot it
sim_animation(sim, T=150)

In [ ]:
str(cuda.Context.get_cache_config())

'PREFER_NONE'

#### Observation
-   Open boundary conditions reduce reflections of outgoing waves at the boundary (but numerically there are still minor reflections)
-   Sponge cells helps to dampen these reflection

## Some Numerical Proofes of Sizes

In [ ]:
eta, hu, hv = sim.download()

print(eta.shape)

(204, 104)


In [ ]:
eta, hu, hv = sim.download(interior_domain_only=True)

print(eta.shape)

(200, 100)


#### Observation 
By "domain" we denominate the area where we intend to solve the SWE
-   The domain has the shape as specified with `nx` and `ny`
-   Ghost cells are added around the domain
-   Sponge cells are part of the domain 